In [74]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

class WebDriver:

	location_data = {}

	def __init__(self):
		self.options = Options()
		self.options.add_argument("--headless")
		self.driver = webdriver.Safari()
		self.location_data["rating"] = "NA"
		self.location_data["reviews_count"] = "NA"
		self.location_data["location"] = "NA"
		self.location_data["contact"] = "NA"
		self.location_data["website"] = "NA"
		self.location_data["Time"] = {"Monday":"NA", "Tuesday":"NA", "Wednesday":"NA", "Thursday":"NA", "Friday":"NA", "Saturday":"NA", "Sunday":"NA"}
		self.location_data["Reviews"] = []
		self.location_data["Popular Times"] = {"Monday":[], "Tuesday":[], "Wednesday":[], "Thursday":[], "Friday":[], "Saturday":[], "Sunday":[]}

	def click_open_close_time(self):

		if(len(list(self.driver.find_elements_by_class_name("cX2WmPgCkHi__section-info-hour-text")))!=0):
			element = self.driver.find_element_by_class_name("cX2WmPgCkHi__section-info-hour-text")
			self.driver.implicitly_wait(5)
			ActionChains(self.driver).move_to_element(element).click(element).perform()

	def click_all_reviews_button(self):

		try:
			WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "allxGeDnJMl__button")))

			element = self.driver.find_element_by_class_name("allxGeDnJMl__button")
			element.click()
		except:
			self.driver.quit()
			return False

		return True

	def get_location_data(self):

		try:
			avg_rating = self.driver.find_element_by_class_name("section-star-display")
			total_reviews = self.driver.find_element_by_class_name("section-rating-term")
			address = self.driver.find_element_by_css_selector("[data-item-id='address']")
			phone_number = self.driver.find_element_by_css_selector("[data-tooltip='Copy phone number']")
			website = self.driver.find_element_by_css_selector("[data-item-id='authority']")
		except:
			pass
		try:
			self.location_data["rating"] = avg_rating.text
			self.location_data["reviews_count"] = total_reviews.text[1:-1]
			self.location_data["location"] = address.text
			self.location_data["contact"] = phone_number.text
			self.location_data["website"] = website.text
		except:
			pass


	def get_location_open_close_time(self):

		try:
			days = self.driver.find_elements_by_class_name("lo7U087hsMA__row-header")
			times = self.driver.find_elements_by_class_name("lo7U087hsMA__row-interval")

			day = [a.text for a in days]
			open_close_time = [a.text for a in times]

			for i, j in zip(day, open_close_time):
				self.location_data["Time"][i] = j
		
		except:
			pass

	def get_popular_times(self):
		try:
			a = self.driver.find_elements_by_class_name("section-popular-times-graph")
			dic = {0:"Sunday", 1:"Monday", 2:"Tuesday", 3:"Wednesday", 4:"Thursday", 5:"Friday", 6:"Saturday"}
			l = {"Sunday":[], "Monday":[], "Tuesday":[], "Wednesday":[], "Thursday":[], "Friday":[], "Saturday":[]}
			count = 0

			for i in a:
				b = i.find_elements_by_class_name("section-popular-times-bar")
				for j in b:
					x = j.get_attribute("aria-label")
					l[dic[count]].append(x)
				count = count + 1

			for i, j in l.items():
				self.location_data["Popular Times"][i] = j
		except:
			pass

	def scroll_the_page(self):
		try:
			WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "section-layout-root")))

			pause_time = 2
			max_count = 5
			x = 0

			while(x<max_count):
				scrollable_div = self.driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show')
				try:
					self.driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
				except:
					pass
				time.sleep(pause_time)
				x=x+1
		except:
			self.driver.quit()

	def expand_all_reviews(self):
		try:
			element = self.driver.find_elements_by_class_name("section-expand-review")
			for i in element:
				i.click()
		except:
			pass

	def get_reviews_data(self):
		try:
			review_names = self.driver.find_elements_by_class_name("section-review-title")
			review_text = self.driver.find_elements_by_class_name("section-review-review-content")
			review_dates = self.driver.find_elements_by_css_selector("[class='section-review-publish-date']")
			review_stars = self.driver.find_elements_by_css_selector("[class='section-review-stars']")

			review_stars_final = []

			for i in review_stars:
				review_stars_final.append(i.get_attribute("aria-label"))

			review_names_list = [a.text for a in review_names]
			review_text_list = [a.text for a in review_text]
			review_dates_list = [a.text for a in review_dates]
			review_stars_list = [a for a in review_stars_final]

			for (a,b,c,d) in zip(review_names_list, review_text_list, review_dates_list, review_stars_list):
				self.location_data["Reviews"].append({"name":a, "review":b, "date":c, "rating":d})

		except Exception as e:
			pass

	def scrape(self, url):
		time.sleep(10)

		self.click_open_close_time()
		self.get_location_data()
		self.get_location_open_close_time()
		self.get_popular_times()
		time.sleep(5)
		self.scroll_the_page()
		self.expand_all_reviews()
		self.get_reviews_data()
		self.driver.quit()

		return(self.location_data)



url = "https://www.google.com/maps/place/Papa+John's+Pizza/@40.7936551,-74.0124687,17z"
x = WebDriver()
print(x.scrape(url))

AttributeError: 'WebDriver' object has no attribute 'find_elements_by_class_name'

In [86]:
from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager
class GoogleMapScraper:

    def __init__(self):
        
        self.driver = webdriver.Edge('/Users/sightes/gitlab/SCRAPPING/GMAP/msedgedriver')
        self.business_list = []
        self.business_info = {}
        self.business_info["name"] = "NA"
        self.business_info["rating"] = "NA"
        self.business_info["reviews_count"] = "NA"
        self.business_info["address"] = "NA"
        self.business_info["contact"] = "NA"
        self.business_info["website"] = "NA"

    def get_business_info(self, url):
        self.driver.get(url)
        # Parse data out of the page
        self.business_info["name"] = self.driver.find_element_by_class_name("x3AX1-LfntMc-header-title-title").text
        self.business_info["rating"] = self.driver.find_element_by_class_name("aMPvhf-fI6EEc-KVuj8d").text
        self.business_info["reviews_count"] = self.driver.find_element_by_class_name("widget-pane-link").text
        self.business_info["address"] = self.driver.find_elements_by_class_name("QSFF4-text")[0].text
        self.business_info["contact"] = self.driver.find_elements_by_class_name("QSFF4-text")[1].text
        self.business_info["website"] = self.driver.find_elements_by_class_name("QSFF4-text")[2].text
        #add business info to business list
        self.business_list.append(self.business_info)

urls = ["https://goo.gl/maps/Kfpy778TpLxj3Nyw6",]
BusinessScraper = GoogleMapScraper()
for url in urls:
    BusinessScraper.get_business_info(url)
print(BusinessScraper.business_info)

/var/folders/cv/gdb475fx4q9by26c926zrqmw0000gn/T/ipykernel_4316/3226283635.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Edge('/Users/sightes/gitlab/SCRAPPING/GMAP/msedgedriver')


AttributeError: 'WebDriver' object has no attribute 'find_element_by_class_name'

In [102]:
from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager
driver = webdriver.Edge('/Users/sightes/gitlab/SCRAPPING/GMAP/msedgedriver')
driver.implicitly_wait(100)
driver.get("https://www.google.cl/maps/search/mayorista10/@-33.2216096,-70.6780596,14z")
driver.maximize_window()
business_list = []
business_info = {}
business_info["name"] = "NA"
business_info["rating"] = "NA"
business_info["reviews_count"] = "NA"
business_info["address"] = "NA"
business_info["contact"] = "NA"
business_info["website"] = "NA"

driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button').click()
time.sleep(1)
driver.find_element_by_xpath("//li[@data-index='1']").click()
time.sleep(5)
driver.find_element(By.CSS_SELECTOR,"[data-item-id='authority']")


/var/folders/cv/gdb475fx4q9by26c926zrqmw0000gn/T/ipykernel_4316/4088538244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge('/Users/sightes/gitlab/SCRAPPING/GMAP/msedgedriver')


AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'